In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import AutoModel

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")  # Use a summarization model

# Create a summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)


In [ ]:
def summarize_text(input_string):
    """
    Summarizes the input string using the specified model.
    
    Args:
        input_string (str): The text to be summarized.
        
    Returns:
        str: The summary of the input string.
    """
    try:
        # Summarize the input string
        summary = summarizer(input_string, max_length=130, min_length=30, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return str(e)




In [ ]:
# Example usage
input_string = """UNIVERSITY OF DELHI
THE UNDERMENTIONED CANDIDATES OF B.E. EIGHTH SEMESTER EXAMINATION HELD IN
MAY/JUNE, 2017 HAVING PASSED THE REMAINING SEMESTER EXAMINATION HELD IN NOV,
2017, THUS HAVING PASSED ALL THE SEMESTERS EXAMINATIONS, THE UNDER
MENTIONED REREGISTERED CANDIDATES IS DECLARED TO HAVE PASSED THE
EXAMINATION FOR THE DEGREE OF BACHELOR OF ENGINEERING (B.E.):-
NETAJI SUBHAS INSTITUTE OF TECHNOLOGY
B.E. (MANUFACTURING PROCESSES AND AUTOMATION ENGINEERING)
ROLLNO NAME AVERAGE CREDITS CLASS
EARNED AWARDED
668MP13 VANSH MENDIRATTA 59.85 228 SECOND CLASS    
AK
AN)
CONTROELER OF EXAMINATIONS
tp
 
- or
FACULTY OF TECHNOLOGY
patep:- | 5 AUG 2024
Owe FA.
Scanned with CamScanner
s.NO/NOT/2024/.4! 7 sade
UNIVERSITY OF DELHI
THE UNDERMENTIONED CANDIDATES OF B.E. EIGHTH SEMESTER EXAMINATION HELD IN
MAY/JUNE, 2005 HAVING PASSED THE REMAINING SEMESTER EXAMINATION HELD IN
MARCH, 2023, THUS HAVING PASSED ALL THE SEMESTERS EXAMINATIONS, THE UNDER
MENTIONED REREGISTERED CANDIDATES IS DECLARED TO HAVE PASSED THE
EXAMINATION FOR THE DEGREE OF BACHELOR OF ENGINEERING (B.E.):-
NETAJI SUBHAS INSTITUTE OF TECHNOLOGY / PRIVATE    
B.E. INSTRUMENTATION AND CONTROL ENGINEERING)      
ROLLNO NAME AVERAGE CREDITS CLASS
EARNED AWARDED
460IC2K 1 PRASHANT BHORIA 59.85 228 SECOND CLASS   
#RK
A
FACULTY

OF TEYHNOLOGY CONTROLL F EXAMINATIONS
patep: 9 5 AUG 2024
dan
Scanned with CamScanner"""
print(summarize_text(input_string))

TALKKING TO MODEL


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the pre-trained DialoGPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


In [13]:
# Initialize chat history
chat_history = []


In [15]:
def chat_with_model(user_input):
    """
    Generates a response to the user's input using DialoGPT.
    
    Args:
        user_input (str): The user's input.
        
    Returns:
        str: The model's response.
    """
    # Encode the user's input and append it to the chat history
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history, new_user_input_ids], dim=-1) if chat_history else new_user_input_ids

    # Generate a response from the model
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Extract and decode the model's response
    chat_history_ids = chat_history_ids[:, bot_input_ids.shape[-1]:]
    bot_response = tokenizer.decode(chat_history_ids[0], skip_special_tokens=True)

    # Update chat history
    chat_history = chat_history_ids

    return bot_response


In [16]:
# Example conversation
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Bot: Goodbye!")
        break
    response = chat_with_model(user_input)
    print(f"Bot: {response}")

UnboundLocalError: cannot access local variable 'chat_history' where it is not associated with a value